# Borrower default risk analysis

This project consists of preparing a report for the loan division of a bank. I am going to find out if the marital status, the number of children, the total income etc. of a customer have an impact on a loan default. The bank already has some data on the creditworthiness of customers.

This report aims to create a **credit score** for a potential customer. A **credit score** is used to assess a potential borrower's ability to repay their loan.

* [Intro](#intro)
* [Stage 1. Data description](#data_review)
    * [1.1 Data collection](#data_collection)
    * [1.2 Data exploration](#data_exploration)
* [Stage 2. Data preprocessing](#data_preprocessing)
    * [2.1 Data preparation](#data_preparation)
    * [2.2 Data_classification](#data_classification)
* [Stage 3. Hypothesis testing](#Hypothesis_testing)
    * [3.1 Data correlation](#data_correlation)
    * [3.2 Credit score creation](#credit_score_creation)
* [General conclusion](#conclusion)

## Intro <a id='intro'></a>

This work aim to analyze the borrower default risk using a set of data of previous customers. We are going to prepare a report for the loan division of a bank to assess a potential borrower's ability to pay back his loan. More precisely we will find out the following:
- link between having children and solvency
- link between marital status and solvency
- link between total income and solvency
- link between purpose and solvency and others.

## Stage 1. Data description <a id='data_review'></a>

### 1.1 Data collection <a id='data_collection'></a>

In [1]:
# Loading all libraries
import pandas as pd
import jinja2

In [2]:
# Loading the data
df = pd.read_csv('credit_scoring_eng.csv')

In [3]:
# Show the first lines of dataframe
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### 1.2 Data exploration <a id='data_exploration'></a>

**Data description**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - the age of the client in years
- `education` - customer education level
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - family status identifier
- `gender` - gender of the client
- `income_type` - employment type
- `debt` - was there any debt in repaying a loan?
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [4]:
# Let's see how many rows and columns our dataset has
df.shape

(21525, 12)

The initial dataframe is 21525 rows and 12 columns

In [5]:
# Let's display the first 10 rows
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


First thing that jumps to the eyes is the values in the column "days_employed": they are negative and this need further investigation. Moreover in the 'education' column we see lower and uppercase characters. We will fix it 

In [6]:
# Get insights into data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In the Dataframe some columns (days_employed, total_income) are not completely filled. For the moment, we leave that like this and soon we will evaluate if there's need to replace the "null values" or just dump them.

In [7]:
# Let's look at the filtered table with missing values from the first column where data is missing
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


We notice that there are 2174 absent values in "days_employed" and we realize that for every row in which "days_employed" is absent, "total_income" is absent too.

In [8]:
# Let's apply multiple conditions to filter data and see the number of rows in the filtered table.
filtro_days_employed = df['days_employed'].isna()
filtro_total_income = df['total_income'].isna()
bank_list = df[filtro_days_employed & filtro_total_income]

len(bank_list)


2174

**Conclusion**

We see that the number of filtered values, that are the rows in which both 'days_employed' and 'total_income' are absent, is equal to the null values in the original dataframe. So, we can assume that there is a link between 'days_employed' and 'total_income'.

In [9]:
# Let's see what percentage of rows with missing values are out of the total data frame
perce =len(bank_list)/len(df)
print(f'Percentage of number of filtered values over the original dataframe values: {perce:.0%}')

Percentage of number of filtered values over the original dataframe values: 10%


The percentage of the rows with missing values over the rows of the initial dataframe is +/- 10%.
I do not see any relation between the absence of values and some specific feature in the row.

Since 10% don't seem to me a scary value, I will compare the relation of every feature (column) with the original dataframe and the dataframe composed only by rows with missing values

In [10]:
# We are going to investigate the clients that do not have data on the identified characteristic and the column with the missing values
print("Relation with 'n of children' before filtering null values")
print(df['children'].value_counts())
print()
print("Relation with 'n of children' before filtering null values (percentage of the total)")
print(100*df['children'].value_counts()/df['children'].count())
print()


Relation with 'n of children' before filtering null values
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Relation with 'n of children' before filtering null values (percentage of the total)
 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64



In [11]:
print("Relation with 'n of children' with null values filtering")
print(bank_list['children'].value_counts())
print()
print("Relation with 'n of children' with null values filtering (percentage of the total)")
print(100*bank_list['children'].value_counts()/bank_list['children'].count())

Relation with 'n of children' with null values filtering
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

Relation with 'n of children' with null values filtering (percentage of the total)
 0     66.191352
 1     21.849126
 2      9.383625
 3      1.655934
 20     0.413983
 4      0.321987
-1      0.137994
 5      0.045998
Name: children, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the n of children and the absent values

Lets do the same with age:

In [12]:
print("Relation with 'age' before filtering null values")
print(df['dob_years'].value_counts().head())
print()
print("Relation with 'age' before filtering null values (percentage of the total)")
print((100*df['dob_years'].value_counts()/df['dob_years'].count()).head())

Relation with 'age' before filtering null values
35    617
40    609
41    607
34    603
38    598
Name: dob_years, dtype: int64

Relation with 'age' before filtering null values (percentage of the total)
35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
Name: dob_years, dtype: float64


In [13]:
print("Relation with 'age' with null values filtering")
print(bank_list['dob_years'].value_counts().head())
print()
print("Relation with 'age' with null values filtering (percentage of the total)")
print((100*bank_list['dob_years'].value_counts()/bank_list['dob_years'].count()).head())

Relation with 'age' with null values filtering
34    69
40    66
42    65
31    65
35    64
Name: dob_years, dtype: int64

Relation with 'age' with null values filtering (percentage of the total)
34    3.173873
40    3.035879
42    2.989880
31    2.989880
35    2.943882
Name: dob_years, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the age and the absent values

Lets do the same with education:

In [14]:
print("Relation with 'education' before filtering null values")
print(df['education'].value_counts())
print()
print("Relation with 'education' before filtering null values (percentage of the total)")
print((100*df['education'].value_counts()/df['education'].count()))

Relation with 'education' before filtering null values
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

Relation with 'education' before filtering null values (percentage of the total)
secondary education    63.879210
bachelor's degree      21.918699
SECONDARY EDUCATION     3.586527
Secondary Education     3.303136
some college            3.103368
BACHELOR'S DEGREE       1.272938
Bachelor's Degree       1.245064
primary education       1.161440
Some College            0.218351
SOME COLLEGE            0.134727
PRIMARY EDUCATION       0.078978
Primary Education       0.069686
gr

In [15]:
print("Relation with 'education' with null values filtering")
print(bank_list['education'].value_counts())
print()
print("Relation with 'education' with null values filtering (percentage of the total)")
print((100*bank_list['education'].value_counts()/bank_list['education'].count()))

Relation with 'education' with null values filtering
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64

Relation with 'education' with null values filtering (percentage of the total)
secondary education    64.765409
bachelor's degree      22.815087
SECONDARY EDUCATION     3.081877
Secondary Education     2.989880
some college            2.529899
Bachelor's Degree       1.149954
BACHELOR'S DEGREE       1.057958
primary education       0.873965
Some College            0.321987
SOME COLLEGE            0.321987
Primary Education       0.045998
PRIMARY EDUCATION       0.045998
Name: education, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the education and the absent values

Lets do the same with family status:

In [16]:
print("Relation with 'family status' before filtering null values")
print(df['family_status'].value_counts())
print()
print("Relation with 'family status' before filtering null values (percentage of the total)")
print((100*df['family_status'].value_counts()/df['family_status'].count()))

Relation with 'family status' before filtering null values
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Relation with 'family status' before filtering null values (percentage of the total)
married              57.514518
civil partnership    19.405343
unmarried            13.068525
divorced              5.551684
widow / widower       4.459930
Name: family_status, dtype: float64


In [17]:
print("Relation with 'family status' with null values filtering")
print(bank_list['family_status'].value_counts())
print()
print("Relation with 'family status' with null values filtering (percentage of the total)")
print((100*bank_list['family_status'].value_counts()/bank_list['family_status'].count()))

Relation with 'family status' with null values filtering
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

Relation with 'family status' with null values filtering (percentage of the total)
married              56.899724
civil partnership    20.331187
unmarried            13.247470
divorced              5.151794
widow / widower       4.369825
Name: family_status, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the family status and the absent values



Lets do the same with gender:

In [18]:
print("Relation with 'gender' before filtering null values")
print(df['gender'].value_counts())
print()
print("Relation with 'gender' before filtering null values (percentage of the total)")
print((100*df['gender'].value_counts()/df['gender'].count()))

Relation with 'gender' before filtering null values
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Relation with 'gender' before filtering null values (percentage of the total)
F      66.137050
M      33.858304
XNA     0.004646
Name: gender, dtype: float64


In [19]:
print("Relation with 'gender' with null values filtering")
print(bank_list['gender'].value_counts())
print()
print("Relation with 'gender' with null values filtering (percentage of the total)")
print((100*bank_list['gender'].value_counts()/bank_list['gender'].count()))

Relation with 'gender' with null values filtering
F    1484
M     690
Name: gender, dtype: int64

Relation with 'gender' with null values filtering (percentage of the total)
F    68.26127
M    31.73873
Name: gender, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the gender and the absent values

Lets do the same with income type:

In [20]:
print("Relation with 'income_type' before filtering null values")
print(df['income_type'].value_counts())
print()
print("Relation with 'income_type' before filtering null values (percentage of the total)")
print((100*df['income_type'].value_counts()/df['income_type'].count()))

Relation with 'income_type' before filtering null values
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Relation with 'income_type' before filtering null values (percentage of the total)
employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
unemployed                      0.009292
entrepreneur                    0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64


In [21]:
print("Relation with 'income_type' with null values filtering")
print(bank_list['income_type'].value_counts())
print()
print("Relation with 'income_type' with null values filtering (percentage of the total)")
print((100*bank_list['income_type'].value_counts()/bank_list['income_type'].count()))

Relation with 'income_type' with null values filtering
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Relation with 'income_type' with null values filtering (percentage of the total)
employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the income type and the absent values

Lets do the same with debt:

In [22]:
print("Relation with 'debt' before filtering null values")
print(df['debt'].value_counts())
print()
print("Relation with 'debt' before filtering null values (percentage of the total)")
print((100*df['debt'].value_counts()/df['debt'].count()))

Relation with 'debt' before filtering null values
0    19784
1     1741
Name: debt, dtype: int64

Relation with 'debt' before filtering null values (percentage of the total)
0    91.911731
1     8.088269
Name: debt, dtype: float64


In [23]:
print("Relation with 'debt' with null values filtering")
print(bank_list['debt'].value_counts())
print()
print("Relation with 'debt' with null values filtering (percentage of the total)")
print((100*bank_list['debt'].value_counts()/bank_list['debt'].count()))

Relation with 'debt' with null values filtering
0    2004
1     170
Name: debt, dtype: int64

Relation with 'debt' with null values filtering (percentage of the total)
0    92.180313
1     7.819687
Name: debt, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the debt and the absent values

Lets do the same with purpose:

In [24]:
print("Relation with 'purpose' before filtering null values")
print(df['purpose'].value_counts().head())
print()
print("Relation with 'purpose' before filtering null values (percentage of the total)")
print((100*df['purpose'].value_counts()/df['purpose'].count()).head())

Relation with 'purpose' before filtering null values
wedding ceremony              797
having a wedding              777
to have a wedding             774
real estate transactions      676
buy commercial real estate    664
Name: purpose, dtype: int64

Relation with 'purpose' before filtering null values (percentage of the total)
wedding ceremony              3.702671
having a wedding              3.609756
to have a wedding             3.595819
real estate transactions      3.140534
buy commercial real estate    3.084785
Name: purpose, dtype: float64


In [25]:
print("Relation with 'purpose' with null values filtering")
print(bank_list['purpose'].value_counts().head())
print()
print("Relation with 'purpose' with null values filtering (percentage of the total)")
print((100*bank_list['purpose'].value_counts()/bank_list['purpose'].count()).head())

Relation with 'purpose' with null values filtering
having a wedding                92
to have a wedding               81
wedding ceremony                76
construction of own property    75
housing transactions            74
Name: purpose, dtype: int64

Relation with 'purpose' with null values filtering (percentage of the total)
having a wedding                4.231831
to have a wedding               3.725851
wedding ceremony                3.495860
construction of own property    3.449862
housing transactions            3.403864
Name: purpose, dtype: float64


The relation between the 2 cases is coherent. So, we can confirm there's no actual connection between the purpose and the absent values

So, the comparation between the dataframe with filtered valued and the initial dataframe brings us to say that the relation between the 1st, 2nd and 3rd values are pretty similar for all the features.

Anyway, I do not think the absent values are due to something in particular. This means their absence is aleatory.

## Stage 2. Data preprocessing <a id='data_preprocessing'></a>

### 2.1 Data preparation <a id='data_preparation'></a>

In [26]:
# Let's look at all the values in the 'education' column to see if the spelling will need to be corrected and what exactly will need to be corrected
df['education'].sort_values().unique()

array(["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE',
       'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education',
       'Some College', "bachelor's degree", 'graduate degree',
       'primary education', 'secondary education', 'some college'],
      dtype=object)

Concerning the 'education' column, there many values that indicate the same thing, so we take care about it in the following

In [27]:
df['education'] = df['education'].str.lower()

In [28]:
# Checking all the values in the column to make sure we have corrected them
df['education'].sort_values().unique()

array(["bachelor's degree", 'graduate degree', 'primary education',
       'secondary education', 'some college'], dtype=object)

In [29]:
# Let's see the distribution of values in the `children` column
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

There are some anomalies in this column:
- 20 children seems absurd
- -1 children is not acceptable

In my opinion, they are both error of typing: '20' means '2' and '-1' means '1'

So, I arrange the new dataframe with both corrections


In [30]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1

In [31]:
# Check the `children` column again to make sure everything is fixed
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Just keep in mind that we are making an assumption about the nature of the errors and it is not wrong in this case to assume that it is a typo, but these modifications with a larger volume of data can alter the behavior of the information. On another occasion, it would be necessary to discuss with those in charge of collecting information.

Now we are going to see if there is some problem among the column of age

In [32]:
df['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

In the 'dob_years' column the only thing that is not acceptable is 0 (zero), so we replace this value with the median of the column without 0 (zero).

In [33]:
median_age = df.loc[df['dob_years'] != 0, 'dob_years'].median()
median_age

43.0

Lets now assign this median value to it:

In [34]:
df.loc[df['dob_years'] == 0, 'dob_years'] = median_age

We verify:

In [35]:
df['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75], dtype=int64)

####  Restore missing values in `days_employed`

Let's work now on the column 'days_employed'

In [36]:
# Find problematic data in `days_employed`, if it exists, re-arrange them
df['days_employed'].head(20)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
Name: days_employed, dtype: float64

We see that in the column 'days_employed' there are negative and float values. Anyway the positive values are one or more order of magnitude higher maybe because of a problem with the collection of 
information. I will replace temporarely all the positive data in 'days_employed' with null-value and later I substitute these null values with the mean or median value of the respective category (to be chosen which one)

In [37]:
df.loc[df['days_employed'] > 0, 'days_employed'] = None

I check if the replacement effectively took place, counting the positive values of column 'days_employed'

In [38]:
len(df[df.days_employed > 0])

0

Ok! we now have no positive value. Let's now convert the neg values to positive using their absolute value

In [39]:
df['days_employed'] = df['days_employed'].abs()

In [40]:
df['days_employed'].describe()

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

Since the distribution of values in 'days_employed' is skewed and there are some outliers(the max is >10x the median value), I think it is better to use the median value to replace the null values. Let's dive into it

To fill the absent values in 'days_employed' we will use the median value of the corresponding age category of the row. But to do this, we need first to find out these categories.

In [41]:
# Let's write a function that calculates the age category
def age_group(age):
    """
    The function returns the age category according to the age value, using the following rules:
    — 18-35 yo for age between 18 and 35 year old
    — 36-46 yo for age between 36 and 46 year old
    — 47-66 yo for age between 47 and 66 year old
    — >66 yo for age over 66 year old
    """
    
    if 18 <= age <= 35:
        return "18-35 yo"
    if 36 <= age <= 46:
        return "36-46 yo"
    if 47 <= age <= 66:
        return "47-66 yo"
    if age > 66:
        return ">66 yo"

Now that we have the category of age we add a new column 'age_cat' referring the age of every customer

In [42]:
# Create a new column 'age_cat' based on the function
df['age_cat'] = df['dob_years'].map(age_group)

Let's now review the distribution of 'days_employed' by age_cat

In [43]:
age_days_employed = df.groupby(['age_cat']).agg({'days_employed' : ['mean', 'median', 'min', 'max']})
age_days_employed

days_employed                                       
                  mean       median         min           max
age_cat                                                      
18-35 yo   1539.404299  1202.380896   24.141633   7622.710756
36-46 yo   2504.763759  1877.007970   33.520665  12506.318296
47-66 yo   3235.292944  2302.063575   30.195337  18388.949901
>66 yo     4024.973149  2598.981129  214.343991  14747.887934

It makes sense because the older a client is, the more days he (or she) has worked.

And now we will replace the null values with the median value of the corresponding age_cat. This because the values are highly skewed (see for example min and max)

In [44]:
def my_lambda(x):
    return x.fillna(x.median())

df['days_employed'] = df.groupby(['age_cat'])['days_employed'].transform(lambda x: my_lambda(x))

To simplify things up, lets take the int of every value.

In [45]:
df['days_employed'] = df['days_employed'].astype(int)

Ok ! Lets see the resulting column info

In [46]:
df['days_employed'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 21525 entries, 0 to 21524
Series name: days_employed
Non-Null Count  Dtype
--------------  -----
21525 non-null  int32
dtypes: int32(1)
memory usage: 84.2 KB


Ok ! we have now 21525 int values in 'days_employed'

Lets analyze now the family_status column to see if there's some anomaly

In [47]:
df['family_status'].sort_values().unique()

array(['civil partnership', 'divorced', 'married', 'unmarried',
       'widow / widower'], dtype=object)

There's no anomaly inside the 'family_status' column, so we skip this.

Lets analyze now the 'gender' column to see if there's some anomaly

In [48]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

There's no anomaly inside the 'gender' column, so we skip this.

Lets analyze now the 'income_type' column to see if there's some anomaly

In [49]:
df['income_type'].sort_values().unique()

array(['business', 'civil servant', 'employee', 'entrepreneur',
       'paternity / maternity leave', 'retiree', 'student', 'unemployed'],
      dtype=object)

There's no anomaly inside the 'income_type' column, so we skip this.

Lets see if there are duplicates:

In [50]:
df.duplicated().value_counts()

False    21454
True        71
dtype: int64

We have 71 duplicated rows and then we should get rid of them and reset the index 

In [51]:
df = df.drop_duplicates().reset_index(drop = True)

In [52]:
# Última comprobación para ver si tenemos duplicados

df.duplicated().value_counts()

False    21454
dtype: int64

Finally we have no duplicated values. Good! Now reset the index 

In [53]:
# Checking the size of the dataset we now have, after having run these first few manipulations
df.shape

(21454, 13)

Filtering the duplicated rows we end up with 21454 rows, so just 0.4% less rows. This does not stop us to proceed.

Restoring missing values in `total_income`

As already stated, the second column with missing values is 'total_income'

In [54]:
# Distribution of mean and median value of `total_income`
df['total_income'].describe().astype(int)

count     19351
mean      26787
std       16475
min        3306
25%       16488
50%       23202
75%       32549
max      362496
Name: total_income, dtype: int32

Since the distribution of values in 'total_income' is skewed and there are some outliers, I think it is better to use the median value to replace the null values. But, as already done with 'days_employed' not just the overall median but the median correspondent to the level of education. Let's dive into it

In [55]:
education_total_income = df.groupby(['education']).agg({'total_income' : ['mean', 'median', 'min', 'max']})
education_total_income

total_income                                   
                             mean      median        min         max
education                                                           
bachelor's degree    33142.802434  28054.5310   5148.514  362496.645
graduate degree      27960.024667  25161.5835  15800.399   42945.794
primary education    21144.882211  18741.9760   4049.374   78410.774
secondary education  24594.503037  21836.5830   3306.762  276204.162
some college         29045.443644  25618.4640   5514.581  153349.533

It makes sense because the higher is the level of study, the higher is the total_income

And now we will replace the null values with the median value of the corresponding education.

In [56]:
def my_lambda(x):
    return x.fillna(x.median())

df['total_income'] = df.groupby(['education'])['total_income'].transform(lambda x: my_lambda(x))

In [57]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Cool ! Now we have filled the null values in 'total_income' with the median value of correspondent education

To simplify things up, lets take the int of every value.

In [58]:
df['total_income'] = df['total_income'].astype(int)

Ok ! Lets see the resulting column info

In [59]:
df['total_income'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 21454 entries, 0 to 21453
Series name: total_income
Non-Null Count  Dtype
--------------  -----
21454 non-null  int32
dtypes: int32(1)
memory usage: 83.9 KB


Ok ! we have now 21525 int values in 'total_income'

Let's now analyze the 'total income' mean and median values based on every column

In [60]:
# Relation between 'age_cat' and 'total_income'
df.groupby(['age_cat']).agg({'total_income': ['mean', 'median']}).astype(int)

total_income       
                 mean median
age_cat                     
18-35 yo        26563  23242
36-46 yo        28207  24031
47-66 yo        25420  21836
>66 yo          20864  18811

In [61]:
# Relation between 'children' and 'total_income'
df.groupby(['children']).agg({'total_income': ['mean', 'median']}).astype(int)


total_income       
                 mean median
children                    
0               26125  22267
1               27022  23164
2               27100  22584
3               28688  23675
4               26661  24522
5               26664  26953

In [62]:
# Relation between 'education' and 'total_income'
df.groupby(['education']).agg({'total_income': ['mean', 'median']}).astype(int)

total_income       
                            mean median
education                              
bachelor's degree          32624  28054
graduate degree            27959  25161
primary education          20965  18741
secondary education        24325  21836
some college               28727  25618

This last, the relation between 'education' and 'total_income', is the more explicit.
In short, the more people are educated, the bigger is their income.
The scale is (from lower to high): 
- primary edu
- secondary edu
- some college 
- grad degree
- BSc degree

In [63]:
# Relation between 'family_status' and 'total_income'
df.groupby(['family_status']).agg({'total_income': ['mean', 'median']}).astype(int)

total_income       
                          mean median
family_status                        
civil partnership        26363  22412
divorced                 26827  22857
married                  26694  22806
unmarried                26660  22864
widow / widower          22939  21617

In [64]:
# Relation between 'gender' and 'total_income'
df.groupby(['gender']).agg({'total_income': ['mean', 'median']}).astype(int)

total_income       
               mean median
gender                    
F             24540  21836
M             30213  26145
XNA           32624  32624

Males are definitive higher earner than female customers

In [65]:
# Relation between 'income_type' and 'total_income'
df.groupby(['income_type']).agg({'total_income': ['mean', 'median']}).astype(int)

total_income       
                                    mean median
income_type                                    
business                           31587  27129
civil servant                      27062  23718
employee                           25575  22125
entrepreneur                       53960  53960
paternity / maternity leave         8612   8612
retiree                            22019  20471
student                            15712  15712
unemployed                         21014  21014

This last as well, the relation between 'income_type' and 'total_income', is reasonable. Business and entrepreneur (maybe are the same) score the highest income while students and paternity leave have the lowest

The 'education' index is, in my opinion, the most trustable index to foresee the total income.
In short, the more people are educated, the bigger is their income.

In [66]:
# Comprobar el número de entradas en las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int32 
 11  purpose           21454 non-null  object
 12  age_cat           21454 non-null  object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.0+ MB


Ok, now we are sure that every column has all non-null values.

### 2.2 Data classification <a id='data_classification'></a>

Let's first arrange the categorical data: 'purpose' column

In [67]:
# Displays the data values selected for classification
df.purpose.sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

We can reduce all these different purposes to just 4. Let's see how

In [68]:
def replace_wrong_purpose(wrong_purposes, correct_purpose):
    for wrong_purpose in wrong_purposes:
        df['purpose'] = df['purpose'].replace(wrong_purpose, correct_purpose)
        
duplicates_1 = ['building a property', 'building a real estate', 'buy commercial real estate', 'buy real estate', 'buy residential real estate', 'buying property for renting out', 
                'construction of own property', 'housing', 'housing renovation', 'housing transactions', 'property', 'purchase of my own house', 'purchase of the house', 
                'purchase of the house for my family', 'real estate transactions', 'transactions with commercial real estate','transactions with my real estate']
correct_1 = "real estate"

duplicates_2 = ['buying a second-hand car', 'buying my own car', 'car', 'car purchase', 'cars', 'purchase of a car', 'second-hand car purchase', 'to buy a car', 'to own a car']
correct_2 = "car purchase"

duplicates_3 = ['education', 'getting an education', 'getting higher education', 'going to university', 'profile education', 'supplementary education', 'to become educated',
                'to get a supplementary education', 'university education']
correct_3 = "education"

duplicates_4 = ['having a wedding', 'to have a wedding', 'wedding ceremony']
correct_4 = "wedding"

replace_wrong_purpose(duplicates_1, correct_1)  
replace_wrong_purpose(duplicates_2, correct_2)
replace_wrong_purpose(duplicates_3, correct_3)
replace_wrong_purpose(duplicates_4, correct_4)

Let's check again for unique values

In [69]:
df.purpose.sort_values().unique()

array(['car purchase', 'education', 'real estate', 'wedding'],
      dtype=object)

Cool, we have now only four kind of purposes instead of the long messy list of before

## Stage 3. Hypothesis testing <a id='Hypothesis_testing'></a>

### 3.1 Data correlations <a id='data_correlation'></a>

**Is there a correlation between having children and paying on time?**

In [70]:
# Checking the data on the children and punctual payments

deb_clients = df['children'].value_counts()

children_deb = df.groupby('children').agg({'debt':'sum'})

children_deb["default_rate_%"] = (100*children_deb['debt']/deb_clients).round(2)

children_deb.sort_values(by = 'default_rate_%', ascending=False, inplace=True)


children_deb.style.bar(color= '#ff6200')

,debt,default_rate_%
children,,
4,4,9.760000
2,202,9.490000
1,445,9.170000
3,27,8.180000
0,1063,7.540000
5,0,0.000000


This last means respectively that:
- 1063 clients with no children have not paid previous debts
- 445 clients with 1 children have not paid previous debts
- etc.

**Conclusion**

So we realize that, statistically speaking, the default rate of customer with no children is the lowest (i did not consider clients with 5 children) thus they are more likely to pay back their debt whereas customers with 4 children are the worst. 

**Is there a correlation between family situation and paying on time?**

In [71]:
# Comprueba los datos del estado familiar y los pagos a tiempo 

fam_stat_clients = df['family_status'].value_counts()

fam_stat_deb = df.groupby('family_status').agg({'debt':'sum'})

fam_stat_deb["default_rate %"] = (100*fam_stat_deb['debt']/fam_stat_clients).round(2)

fam_stat_deb.sort_values(by = 'default_rate %', ascending=False, inplace=True)

fam_stat_deb.style.bar(color= '#ff6200')

,debt,default_rate %
family_status,,
unmarried,274,9.750000
civil partnership,388,9.350000
married,931,7.550000
divorced,85,7.110000
widow / widower,63,6.570000


**Conclusion**

So we realize that the default rate of widow customer is the lowest thus they are more likely to pay back their debt whereas unmarried customers are the worst. 

**Is there a correlation between income level and on-time payment?**

To do this, let's create a category of different income levels

In [72]:
def total_income_group(income):
    """
    The function returns the total income, using the following rules:
    — >50k for customer who get more than 50k/year
    — 35-50k for customer who get between 35k and 50k/year
    — 25-34k for customer who who get between 25k and 34k/year
    — 15-24k for customer who who get between 15k and 24k/year
    — <15h for customer who who get less than 15k/year    
    """
    
    if income > 50000:
        return '>50k'
    if 35000 < income <= 50000:
        return '35-50k'
    if 25000 < income <= 35000:
        return '25-34k'
    if 15000 < income <= 25000:
        return '15-24k'
    if income <= 15000:
        return '<15k'   

In [73]:
df['income_cat'] = df['total_income'].map(total_income_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_cat,income_cat
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,real estate,36-46 yo,35-50k
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,36-46 yo,15-24k
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,real estate,18-35 yo,15-24k
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,education,18-35 yo,35-50k
4,0,2302,53,secondary education,1,civil partnership,1,F,retiree,0,25378,wedding,47-66 yo,25-34k


Good, Now we have a new column with income_cat !

In [74]:
income_clients = df['income_cat'].value_counts()

income_deb = df.groupby('income_cat').agg({'debt':'sum'})

income_deb["default_rate_%"] = (100*income_deb['debt']/income_clients).round(2)

income_deb.sort_values(by = 'default_rate_%', ascending=False, inplace=True)

income_deb.style.bar(color= '#ff6200')


,debt,default_rate_%
income_cat,,
15-24k,739,8.690000
25-34k,414,8.060000
<15k,298,7.960000
35-50k,198,7.210000
>50k,92,6.970000


This last means respectively that:
- 739 clients with total_income between 15k-24k have not paid previous debts
- 414 clients with total_income between 25k-34k have not paid previous debts
- etc.

So we realize that, statistically speaking, customers with higher income are more likely to pay back their debt.

**How does the purpose of the loan affect the default rate?**

In [75]:
# Consulting the default rate percentages for each credit purpose and analyze them

purpose_clients = df['purpose'].value_counts()

purpose_deb = df.groupby('purpose').agg({'debt':'sum'})

purpose_deb["default_rate_%"] = (100*purpose_deb['debt']/purpose_clients).round(2)

purpose_deb.sort_values(by = 'default_rate_%', ascending=False, inplace=True)

purpose_deb.style.bar(color= '#ff6200')

,debt,default_rate_%
purpose,,
car purchase,403,9.360000
education,370,9.220000
wedding,186,8.000000
real estate,782,7.230000


So we realize that, statistically speaking, customers with real estate as a purpose are more likely to pay back their debt and customers with car purchase are the least trustable

**How does education affect the default rate?**

In [76]:
education_clients = df['education'].value_counts()

education_deb = df.groupby('education').agg({'debt':'sum'})

education_deb["default_rate_%"] = (100*education_deb['debt']/education_clients).round(2)

education_deb.sort_values(by = 'default_rate_%', ascending=False, inplace=True)

education_deb.style.bar(color= '#ff6200')

,debt,default_rate_%
education,,
primary education,31,10.990000
some college,68,9.140000
secondary education,1364,8.990000
bachelor's degree,278,5.300000
graduate degree,0,0.000000


So we realize that, statistically speaking, customers with bachelor's degree  are more likely to pay back their debt and customers with primary education are the least trustable

### 3.2 Credit score creation <a id='credit_score_creation'></a>

Now we know for each category evaluated (number of children, family status, total income, purpose and education) which group is more financially reliable and who is less. So we can create an index for every one and then calculate the overall creditworthiness of every customer

Lets begin setting an index for children

In [77]:
def children_index(children):
    """
    The function returns the children index according to the previous analysis, using the following rules:
    — index = 6 (max) for customer with 5 children
    — index = 5 for customer with 0 children
    — index = 4 for customer with 3 children
    — index = 3 for customer with 1 children
    — index = 2 for customer with 2 children
    — index = 1 (min) for customer with 4 children 
    """
    
    if children == 5:
        return 6
    if children == 0:
        return 5
    if children == 3:
        return 4
    if children == 1:
        return 3
    if children == 2:
        return 2
    if children == 4:
        return 1

Let's create a new column 'children_ind' using the previous function

In [78]:
df['children_ind'] = df['children'].map(children_index)

Lets set an index for age

In [79]:
def age_index(age):
    
    """
    The function returns the age index according to the previous analysis, using the following rules:
    — index = 4 (max) for customer aged 18-35
    — index = 3 for customer aged 36-46
    — index = 2 for customer aged 47-66
    — index = 1 (min) for customer aged >66  
    """
    
    if 18 <= age <= 35:
        return 1
    if 36 <= age <= 46:
        return 2
    if 47 <= age <= 66:
        return 3
    if age >= 66:
        return 4

Let's create a new column 'age_ind' using the previous function

In [80]:
df['age_ind'] = df['dob_years'].map(age_index)

Lets set an index for family status

In [81]:
def family_status_index(family_status):
    """
    The function returns the family status index according to the previous analysis, using the following rules:
    — index = 5 (max) for widow customer 
    — index = 4 for divorced customer 
    — index = 3 for married customer
    — index = 2 for civil partnership customer
    — index = 1 (min) for unmarried customer  
    """
    
    if family_status == 'unmarried':
        return 1
    if family_status == 'civil partnership':
        return 2
    if family_status == 'married':
        return 3
    if family_status == 'divorced':
        return 4
    if family_status == 'widow / widower':
        return 5

Let's create a new column 'family_status_ind' using the previous function

In [82]:
df['family_status_ind'] = df['family_status'].map(family_status_index)

Lets set an index for education

In [83]:
def education_index(education):
    """
    The function returns the education index according to the previous analysis, using the following rules:
    — index = 5 (max) for graduate degree customer 
    — index = 4 for bachelor's degree customer 
    — index = 3 for secondary education customer
    — index = 2 for some college customer
    — index = 1 (min) for primary education customer  
    """
    
    if education == 'primary education':
        return 1
    if education == 'some college':
        return 2
    if education == 'secondary education':
        return 3
    if education == "bachelor's degree":
        return 4
    if education == "graduate degree":
        return 5

Let's create a new column 'education_ind' using the previous function

In [84]:
df['education_ind'] = df['education'].map(education_index)

Lets set an index for income

In [85]:
def income_index(income):
    """
    The function returns the income index according to the previous analysis, using the following rules:
    — index = 5 (max) for customer who get more than 50k/year
    — index = 4 for customer who get between 35k and 50k/year
    — index = 3 for customer who who get less than 15k/year
    — index = 2 for customer who who get between 25k and 35k/year
    — index = 1 (min) for customer who between 15k and 24k/year    
    """
    
    if income > 50000:
        return 5
    if 35000 < income <= 50000:
        return 4
    if 25000 < income <= 35000:
        return 2
    if 15000 < income <= 25000:
        return 1
    if income <= 15000:
        return 3 

Let's create a new column 'income_ind' using the previous function

In [86]:
df['income_ind'] = df['total_income'].map(income_index)

Lets set an index for purpose

In [87]:
def purpose_index(purpose):
    """
    The function returns the purpose index according to the previous analysis, using the following rules:
    — index = 4 (max) for real estate customer 
    — index = 3 for wedding customer
    — index = 2 for education customer
    — index = 1 (min) car purchase customer  
    """
    
    if purpose == 'car purchase':
        return 1
    if purpose == 'education':
        return 2
    if purpose == 'wedding':
        return 3
    if purpose == "real estate":
        return 4

Let's create a new column 'purpose_ind' using the previous function

In [88]:
df['purpose_ind'] = df['purpose'].map(purpose_index)

And finally we can have the final index of every row just calculating the mean value of all the indexes

In [89]:
df['final_ind'] = ((df['children_ind'] + df['age_ind'] + df['family_status_ind'] + df['education_ind'] + df['income_ind'] + df['purpose_ind'])/6).round(2)

Practically speaking, the final index ranges between 1 and 4.8 where. The smaller is the final index the lower is the creditworthiness of the debtor.

## General conclusion <a id='conclusion'></a>

We begun with a 21525x12 dataframe whith more or less 10% of rows with absent values. These absent values were always in the columns "days_employed" and "total_income" at the same time but anyway I did not discovered any connection between them.
Once I worked the values in "days_employed", meaning that I deleted the original positive values because to high to be reasonable and replaced them with the median value of the correspondent age category and converted the negative float values into positive int values.
I dropped the duplicated values (71 of them) and filled the absent values "total_income"  with the median value of the correspondent education. I did so because there were some outside values much higher or much smaller that could affect the simple average.
After all this, I ended up with 21454 rows.
After that I ran the analysis of every column based on categories and in relation with the ability to pay back the debt.

Finally I discovered that prime indicators to measure the likeability of a client to pay back his debt are:
- higher education
- real estate purpose 
- high income level
- not having children
- be a widow/widower